In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
import os

In [4]:
#Load
base_path = r"C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\0_raw"
train_file = os.path.join(base_path, "0_merged_train.csv")
train_df=pd.read_csv(train_file)


# Remove outliers

In [3]:
#Check which building has the max meter reading -> outlier remove!
print(train_df[train_df.meter_reading == train_df.meter_reading.max()])

        site_id  building_id primary_use  square_feet  year_built  \
236943       15         1381      Office        84615      1963.0   
236944       15         1381      Office        84615      1963.0   
236945       15         1381      Office        84615      1963.0   
236946       15         1381      Office        84615      1963.0   
236947       15         1381      Office        84615      1963.0   
...         ...          ...         ...          ...         ...   
245392       15         1381      Office        84615      1963.0   
245393       15         1381      Office        84615      1963.0   
245394       15         1381      Office        84615      1963.0   
245395       15         1381      Office        84615      1963.0   
245396       15         1381      Office        84615      1963.0   

        floor_count  meter  meter_reading  air_temperature  cloud_coverage  \
236943          NaN      2         1106.1             -1.0             NaN   
236944         

In [ ]:
train_df.drop(['year_built', 'floor_count'], axis=1,inplace=True)

In [20]:
#check
train_df = train_df.drop(train_df[train_df['building_id'] == 1381].index)
print(train_df[train_df.meter_reading == train_df.meter_reading.max()])

       Unnamed: 0.1  Unnamed: 0  site_id  building_id primary_use  \
17546         17546       17546        2          221   Education   
17547         17547       17547        2          221   Education   
17548         17548       17548        2          221   Education   
17549         17549       17549        2          221   Education   
17550         17550       17550        2          221   Education   
...             ...         ...      ...          ...         ...   
26324         26324       26324        2          221   Education   
26325         26325       26325        2          221   Education   
26326         26326       26326        2          221   Education   
26327         26327       26327        2          221   Education   
26328         26328       26328        2          221   Education   

       square_feet  year_built  floor_count  meter          timestamp_x  ...  \
17546       204349      1993.0          NaN      1  2016-03-22 20:00:00  ...   
17547      

# Missing Value Imputation

In [8]:
def percent_missing_val(df):

  percent_missing = (df.isnull().sum() * 100) / len(df)
  missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
  return missing_value_df


In [4]:
df = percent_missing_val(train_df)
df

,column_name,percent_missing
site_id,site_id,0.000000
building_id,building_id,0.000000
primary_use,primary_use,0.000000
square_feet,square_feet,0.000000
year_built,year_built,63.583614
floor_count,floor_count,63.218391
meter,meter,0.000000
meter_reading,meter_reading,0.000000
air_temperature,air_temperature,0.050323
cloud_coverage,cloud_coverage,40.135720


In [9]:
def nan_fillers(df):
  air_temp_df=df.groupby(['site_id', 'day', 'month'])['air_temperature'].transform('mean')
  df['air_temperature'].fillna(air_temp_df, inplace=True)

  dew_temp_df=df.groupby(['site_id', 'day', 'month'])['dew_temperature'].transform('mean')
  df['dew_temperature'].fillna(dew_temp_df, inplace=True)

  cloud_df=df.groupby(['site_id', 'day', 'month'])['cloud_coverage'].transform('mean')
  df['cloud_coverage'].fillna(cloud_df, inplace=True)

  sea_level_df=df.groupby(['site_id', 'day', 'month'])['sea_level_pressure'].transform('mean')
  df['sea_level_pressure'].fillna(sea_level_df, inplace=True)

  precip_df=df.groupby(['site_id', 'day', 'month'])['precip_depth_1_hr'].transform('mean')
  df['precip_depth_1_hr'].fillna(precip_df, inplace=True)

  wind_dir_df=df.groupby(['site_id', 'day', 'month'])['wind_direction'].transform('mean')
  df['wind_direction'].fillna(wind_dir_df, inplace=True)

  wind_speed_df=df.groupby(['site_id', 'day', 'month'])['wind_speed'].transform('mean')
  df['wind_speed'].fillna(wind_speed_df, inplace=True)


  return df

In [6]:


train_df_clean=nan_fillers(train_df)

In [7]:
train_df_clean['cloud_coverage'].fillna(train_df_clean['cloud_coverage'].median(), inplace=True)
train_df_clean['sea_level_pressure'].fillna(train_df_clean['sea_level_pressure'].median(), inplace=True)
train_df_clean['precip_depth_1_hr'].fillna(train_df_clean['precip_depth_1_hr'].median(), inplace=True)

In [8]:
train_df_clean.to_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\1_interim\1_cleaned_train.csv', index=False)


# TEST

In [5]:
test_file = os.path.join(base_path, "0_merged_test.csv")
test_df=pd.read_csv(test_file)

In [6]:
#drop bad quality features
test_df.drop(['year_built', 'floor_count'], axis=1,inplace=True)



In [10]:
#Missing values and NaN values
test_df=nan_fillers(test_df)
test_df['cloud_coverage'].fillna(test_df['cloud_coverage'].median(), inplace=True)
test_df['sea_level_pressure'].fillna(test_df['sea_level_pressure'].median(), inplace=True)
test_df['precip_depth_1_hr'].fillna(test_df['precip_depth_1_hr'].median(), inplace=True)


In [11]:
#Output
test_df.to_csv(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\1_interim\1_cleaned_test.csv', index=False)


In [12]:
test_df

,site_id,building_id,primary_use,square_feet,row_id,meter,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,dayofweek,month,dayofyear,day,year,timestamp
0,0,58,Lodging/residential,83379,1037617,0,25.0,6.000000,20.0,1.382557,1019.700000,0.0,0.0,3,5,12,336,2,2017,2017-12-02 03:00:00
1,0,58,Lodging/residential,83379,1037617,0,24.4,3.044610,21.1,-1.000000,1020.200000,70.0,1.5,3,5,12,336,2,2017,2017-12-02 03:00:00
2,0,58,Lodging/residential,83379,1037617,0,22.8,2.000000,21.1,0.000000,1020.200000,0.0,0.0,3,5,12,336,2,2017,2017-12-02 03:00:00
3,0,58,Lodging/residential,83379,1037617,0,21.1,2.000000,20.6,0.000000,1020.100000,0.0,0.0,3,5,12,336,2,2017,2017-12-02 03:00:00
4,0,58,Lodging/residential,83379,1037617,0,20.0,2.000000,20.0,-1.000000,1020.000000,250.0,2.6,3,5,12,336,2,2017,2017-12-02 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261907,15,1381,Office,84615,39666699,2,3.0,1.109988,-8.0,3.495009,1016.980579,180.0,5.7,12,0,2,57,26,2018,2018-02-26 12:00:00
261908,15,1381,Office,84615,39666699,2,2.8,2.000000,-8.9,3.495009,1007.400000,180.0,7.7,12,0,2,57,26,2018,2018-02-26 12:00:00
261909,15,1381,Office,84615,39666699,2,2.8,1.109988,-7.2,3.495009,1007.500000,180.0,5.1,12,0,2,57,26,2018,2018-02-26 12:00:00
261910,15,1381,Office,84615,39666699,2,2.2,1.109988,-6.7,3.495009,1008.000000,170.0,4.6,12,0,2,57,26,2018,2018-02-26 12:00:00
